In [1]:
# Importamos librerias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Importamos el dataset raw (Food_Delivery_Times.csv)
df = pd.read_csv("../data/raw.csv")

In [3]:
# Inspección general del dataset
display(df.head())
display(df.tail())
display(df.describe())

,Order_ID,Distance_km,Weather,Traffic_Level,Time_of_Day,Vehicle_Type,Preparation_Time_min,Courier_Experience_yrs,Delivery_Time_min
0,522,7.93,Windy,Low,Afternoon,Scooter,12,1.0,43
1,738,16.42,Clear,Medium,Evening,Bike,20,2.0,84
2,741,9.52,Foggy,Low,Night,Scooter,28,1.0,59
3,661,7.44,Rainy,Medium,Afternoon,Scooter,5,1.0,37
4,412,19.03,Clear,Low,Morning,Bike,16,5.0,68


,Order_ID,Distance_km,Weather,Traffic_Level,Time_of_Day,Vehicle_Type,Preparation_Time_min,Courier_Experience_yrs,Delivery_Time_min
995,107,8.50,Clear,High,Evening,Car,13,3.0,54
996,271,16.28,Rainy,Low,Morning,Scooter,8,9.0,71
997,861,15.62,Snowy,High,Evening,Scooter,26,2.0,81
998,436,14.17,Clear,Low,Afternoon,Bike,8,0.0,55
999,103,6.63,Foggy,Low,Night,Scooter,24,3.0,58


,Order_ID,Distance_km,Preparation_Time_min,Courier_Experience_yrs,Delivery_Time_min
count,1000.000000,1000.000000,1000.000000,970.000000,1000.000000
mean,500.500000,10.059970,16.982000,4.579381,56.732000
std,288.819436,5.696656,7.204553,2.914394,22.070915
min,1.000000,0.590000,5.000000,0.000000,8.000000
25%,250.750000,5.105000,11.000000,2.000000,41.000000
50%,500.500000,10.190000,17.000000,5.000000,55.500000
75%,750.250000,15.017500,23.000000,7.000000,71.000000
max,1000.000000,19.990000,29.000000,9.000000,153.000000


In [4]:
# Verificamos valores nulos
display(df.isnull().sum())

Order_ID                   0
Distance_km                0
Weather                   30
Traffic_Level             30
Time_of_Day               30
Vehicle_Type               0
Preparation_Time_min       0
Courier_Experience_yrs    30
Delivery_Time_min          0
dtype: int64

Dado que existen valores nulos en el DataFrame, realizaremos un conteo total de estos para evaluar su impacto sobre el conjunto de datos. Esta información nos permitirá tomar decisiones informadas sobre el tratamiento más adecuado, ya sea imputación o eliminación, según el grado de afectación.

In [5]:
# Sumaremos la totalidad de los valores nulos
total_nulos = df.isnull().sum().sum()
total_celdas = df.size
porcentaje = (total_nulos / total_celdas) * 100
print(f"Nulos totales: {total_nulos} ({porcentaje:.2f}%)")

Nulos totales: 120 (1.33%)


Durante la inspección de calidad de los datos, se identificaron 120 valores nulos, lo que representa aproximadamente el 1.33% del total de entradas del DataFrame. Este porcentaje es relativamente bajo y no representa una amenaza significativa para la integridad del análisis o los modelos que se construyan posteriormente.

Para asegurar la consistencia de los datos y evitar pérdida de información, se procederá a imputar los valores faltantes utilizando las siguientes estrategias:

Variables numéricas: serán imputadas utilizando la media de cada columna.

Variables categóricas: serán imputadas utilizando la moda, es decir, el valor más frecuente.

Este enfoque permite conservar la mayor cantidad posible de datos sin introducir sesgos importantes, manteniendo así la representatividad del conjunto.

In [6]:
# Imputaremos de manera estadística

# Variables categóricas
df["Weather"] = df["Weather"].fillna(df["Weather"].mode()[0])
df["Traffic_Level"] = df["Traffic_Level"].fillna(df["Traffic_Level"].mode()[0])
df["Time_of_Day"] = df["Time_of_Day"].fillna(df["Time_of_Day"].mode()[0])

# Variables numéricas
df["Courier_Experience_yrs"] = df["Courier_Experience_yrs"].fillna(df["Courier_Experience_yrs"].mean())

In [7]:
# Confirmamos la cantidad de valores nulos en el Dataframe
display(df.isnull().sum())

Order_ID                  0
Distance_km               0
Weather                   0
Traffic_Level             0
Time_of_Day               0
Vehicle_Type              0
Preparation_Time_min      0
Courier_Experience_yrs    0
Delivery_Time_min         0
dtype: int64

In [8]:
# Verificamos valors duplicados
print(f"Total de valores duplicados: {df.duplicated().sum()}")

Total de valores duplicados: 0
